In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate() + LoggerMessageGate())
    
    x_prev = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1  = add!(model, constvar(:add_1, 1.0))
    x      = add!(model, randomvar(:x))
    
    x_prev_add = add!(model, make_node(+, x_prev, add_1, x))
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    y     = add!(model, datavar(:y, Dirac{Float64}))
    
    add_x_and_noise = add!(model, make_node(+, x, noise, y))
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [3]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [5]:
kalman(data)

From variable in2 of node + => Normal{Float64}(μ=-1.0, σ=100.0)
From variable out of node + => Normal{Float64}(μ=16.117435877483537, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=14.781799879885819, σ=14.002800840280099)
From variable out of node + => Normal{Float64}(μ=-18.221915217563225, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=-2.55667294063351, σ=9.950371902099892)
From variable out of node + => Normal{Float64}(μ=0.2697253451065178, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=-2.620779468534163, σ=8.137884587711596)
From variable out of node + => Normal{Float64}(μ=37.60744044950428, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=6.3862404115749065, σ=7.053456158585984)
From variable out of node + => Normal{Float64}(μ=30.921085276151175, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=10.273659707307232, σ=6.311944030978033)
From variable out of node + => 

10-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=15.781799879885819, σ=14.002800840280099)
 Normal{Float64}(μ=-1.55667294063351, σ=9.950371902099892)
 Normal{Float64}(μ=-1.6207794685341632, σ=8.137884587711596)
 Normal{Float64}(μ=7.3862404115749065, σ=7.053456158585984)
 Normal{Float64}(μ=11.273659707307232, σ=6.311944030978033)
 Normal{Float64}(μ=2.5563033646362645, σ=5.763904177042351)
 Normal{Float64}(μ=1.9967810035967433, σ=5.337605126836238)
 Normal{Float64}(μ=2.427899756060979, σ=4.993761694389224)
 Normal{Float64}(μ=2.148649235188182, σ=4.708816093480111)
 Normal{Float64}(μ=2.6749507209247207, σ=4.467670516087703)